# 데이터셋 확인

## 0. 필요 라이브러리 import

In [25]:
#필요한 패키지를 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

#jupyter에서 matplotlib을 사용해 만든 graph를 화면에 표시하기 위해 필요
%matplotlib inline 

#아래는 한글을 사용할 때 깨지는 문제에 대한 해결
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

#그래프의 축 등에서 음수를 표시할 때 minus sign이 깨지는 것 해결
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

plt.rcParams["figure.figsize"] = (12, 12)


## 1. 데이터 불러오기

* 2019년 7월 17일 ~ 2020년 09월 30일까지의 서울, 경기도 업종별 배달 주문건수 및 기상, 환경 코로나 데이터

In [26]:
raw_data = pd.read_csv('data/predict/predict_변수추가최종.csv', encoding='utf-8', index_col=0)
raw_data

,광역시도명,측정날짜,측정일시,PM10,기온,강수량,풍속,습도,운량,적설,확진자수,계절,요일,기념일,공휴일
0,서울,2021-03-19,0,116.240,12.50,0.0,0.80,47.0,9.0,0.0,118,봄,금,0,0
1,서울,2021-03-19,1,114.400,12.40,0.0,1.70,49.0,9.0,0.0,118,봄,금,0,0
2,서울,2021-03-19,2,97.360,12.50,0.0,2.40,52.0,10.0,0.0,118,봄,금,0,0
3,서울,2021-03-19,3,96.800,12.40,0.0,2.60,55.0,10.0,0.0,118,봄,금,0,0
4,서울,2021-03-19,4,90.640,12.20,0.0,1.80,55.0,10.0,0.0,118,봄,금,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,경기도,2021-04-19,19,68.578,15.08,0.0,2.42,55.8,0.0,0.0,180,봄,월,0,0
1532,경기도,2021-04-19,20,67.990,13.62,0.0,2.04,62.0,0.0,0.0,180,봄,월,0,0
1533,경기도,2021-04-19,21,66.196,12.50,0.0,1.80,65.4,1.0,0.0,180,봄,월,0,0
1534,경기도,2021-04-19,22,64.941,11.36,0.0,1.58,71.4,0.0,0.0,180,봄,월,0,0


## 2. 데이터셋 확인

### 1) NULL 값 확인

In [27]:
raw_data.isnull().sum()

광역시도명    0
측정날짜     0
측정일시     0
PM10     0
기온       0
강수량      0
풍속       0
습도       0
운량       0
적설       0
확진자수     0
계절       0
요일       0
기념일      0
공휴일      0
dtype: int64

#### 2-2) 강수/적설 결합
* 강수/적설이 있는 날은 1, 나머지 0인 '눈비' 컬럼 생성

In [28]:
## 눈비 컬럼 생성

raw_data['눈비'] = 0

raw_data.loc[(raw_data['강수량'] > 0) | (raw_data['적설'] > 0), '눈비'] = 1 

In [30]:
raw_data.drop(['적설'], axis=1, inplace=True)


### 3) 파생변수 추가

운량에 따라   
0~2 -> 맑음(1) 3 ~ 5 -> 구름 조금(2) 6 ~ 8 -> 구름 많음(3) 9 ~ 10 -> 흐림(4)

In [31]:
final_data = raw_data.copy()

final_data['날씨'] = 4
final_data.loc[(final_data['운량'] >= 0) & (final_data['운량'] <=2), '날씨'] = 1
final_data.loc[(final_data['운량'] >= 3) & (final_data['운량'] <=5), '날씨'] = 2
final_data.loc[(final_data['운량'] >= 6) & (final_data['운량'] <=8), '날씨'] = 3

In [32]:
final_data.columns

Index(['광역시도명', '측정날짜', '측정일시', 'PM10', '기온', '강수량', '풍속', '습도', '운량', '확진자수',
       '계절', '요일', '기념일', '공휴일', '눈비', '날씨'],
      dtype='object')

In [33]:
final_data.to_csv('data/predict/predict_dummy전.csv', encoding='utf-8')